In [1]:
# need to do this before transformer imports
import os
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'

In [2]:
import os
os.chdir('/workspace/FutureGPT2/src/')
from evals.utils import *
from models.bigram_model import *
from models.mlp_model import *
from models.future_model import *
from data.utils import get_tokenizer
import datasets
from torch.utils.data import DataLoader
from torch import nn
from itertools import islice
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from torch import nn

from tqdm import tqdm
import pandas as pd
import gc
from glob import glob

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
ds_llama = datasets.load_from_disk(f'/workspace/corpus/msmarco/msmarco_LLAMA2_64tokens_1m').with_format('torch', device=torch.device('cuda'))
ds_mistral = datasets.load_from_disk(f'/workspace/corpus/msmarco/msmarco_MISTRAL_64tokens_1m').with_format('torch', device=torch.device('cuda'))
val = {
    'LLAMA2': ds_llama['val'],
    'MISTRAL': ds_mistral['val'],
}

In [6]:
pre = '/workspace/checkpoints/'
layer = 31
cls = 'mlp'
ckpts = {
    'LLAMA2': glob(pre + f'LLAMA2-NECK-SWEEP_*daryl149/llama-2-7b-hf_hidden_idxs-{layer}_hidden_lb-0_token_lb-0_neck_cls-{cls}*')[0],
    'LLAMA2-CHAT': glob(pre + f'LLAMA2-NECK-SWEEP_*daryl149/llama-2-7b-chat-hf_hidden_idxs-{layer}_hidden_lb-0_token_lb-0_neck_cls-{cls}*')[0],
    'LLAMA2-RAND': glob(pre + f'LLAMA2-RAND-INIT*_hidden_idxs-{layer}_hidden_lb-0_neck_cls-{cls}*')[0],
    'MISTRAL': glob(pre + f'MISTRAL-NECK-SWEEP*hidden_idxs-{layer}_hidden_lb-0_token_lb-0_neck_cls-{cls}*')[0],
}
for k, v in ckpts.items():
    print(k)
    print(v)

LLAMA2
/workspace/checkpoints/LLAMA2-NECK-SWEEP_20240117-035543-FBa93_base_model_name-daryl149/llama-2-7b-hf_hidden_idxs-31_hidden_lb-0_token_lb-0_neck_cls-mlp_epoch=00-val_self_loss=3.62.ckpt
LLAMA2-CHAT
/workspace/checkpoints/LLAMA2-NECK-SWEEP_20240117-234516-0ca77_base_model_name-daryl149/llama-2-7b-chat-hf_hidden_idxs-31_hidden_lb-0_token_lb-0_neck_cls-mlp_epoch=00-val_self_loss=3.28.ckpt
LLAMA2-RAND
/workspace/checkpoints/LLAMA2-RAND-INIT_20240129-180219-4a19d_dataset_name-msmarco_hidden_idxs-31_hidden_lb-0_neck_cls-mlp_pretrained-0_token_lb-0_epoch=00-val_self_loss=10.33.ckpt
MISTRAL
/workspace/checkpoints/MISTRAL-NECK-SWEEP_20231231-100744-dDbEa_hidden_idxs-31_hidden_lb-0_token_lb-0_neck_cls-mlp_epoch=00-val_self_loss=4.31.ckpt


In [6]:
necks = {}
for k in ckpts:
    print(k)
    model = LitFutureModelWithNeck.load_from_checkpoint(ckpts[k], strict=False, pretrained=False)
    necks[k] = model.future_neck
    gc.collect()
    torch.cuda.empty_cache()

LLAMA2
LLAMA2-CHAT
LLAMA2-RAND
MISTRAL


In [16]:
# consistent ordering
models = ['LLAMA2', 'LLAMA2-CHAT', 'LLAMA2-RAND', 'MISTRAL']

In [15]:
gc.collect()
torch.cuda.empty_cache()

In [18]:
from itertools import islice 

losses = []
for k in models:
    print('BASE', k)
    model = LitFutureModelWithNeck.load_from_checkpoint(ckpts[k], strict=False)
    base_losses = []
    loader = DataLoader(val[k.split('-')[0]], batch_size=128)
    for kn in models:
        print('NECK', kn)
        model.future_necks = nn.ModuleDict({'default': necks[kn]})
        model.future_neck = model.future_necks['default']
        with torch.no_grad():
            neck_losses = [model._compute_loss(batch).self_loss.item() for batch in islice(iter(loader), 2)]
        print('LOSS', np.mean(neck_losses))
        base_losses.append(np.mean(neck_losses))
    losses.append(base_losses)
    gc.collect()
    torch.cuda.empty_cache()

BASE LLAMA2


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NECK LLAMA2


AttributeError: 'MLPNeck' object has no attribute 'items'

In [ ]:
losses